In [1]:
import os
import glob
from pathlib import Path
import shutil
import json
from google.cloud import storage
import ee

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Authenticate and Initialize ee
ee.Authenticate()
ee.Initialize(project='nsw-dpe-gee-tst')

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/JOSS_data/nsw-dpe-gee-tst-d8aef9f906dc.json"
! gcloud auth login
! gcloud config set project nsw-dpe-gee-tst


You are running on a Google Compute Engine virtual machine.
It is recommended that you use service accounts for authentication.

You can run:

  $ gcloud config set account `ACCOUNT`

to switch accounts if necessary.

Your credentials may be visible to others with access to this
virtual machine. Are you sure you want to authenticate with
your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.

In [5]:
def list_blobs(bucket_name, folder_path):
    storage_client = storage.Client()
    blobs = storage_client.list_blobs(bucket_name, prefix=folder_path)
    blobs = [blob for blob in blobs if blob.name != folder_path and not blob.name.endswith('/')]
    return blobs

def Write_json(new_data, filename):
    with open(filename, 'r+') as file:
        # Load existing data into a dict
        file_data = json.load(file)
        # Write new data
        file_data['name'] = new_data
        # Sets file's current position at offset
        file.seek(0)
        # convert back to json
        json.dump(file_data, file, indent=4)
        file.truncate()
        file.close()

def Write_json_Manifest_MultiTiles(sources, tile, filename):
    with open(filename, 'r+') as file:
        # Load existing data into a dict
        file_data = json.load(file)
        # Write new data
        file_data['tilesets'][0]['sources'] = [sources]
        file_data['properties'] = tile
        # Sets file's current position at offset
        file.seek(0)
        # convert back to json
        json.dump(file_data, file, indent=4)
        file.truncate()
        file.close()

In [6]:
# Buckets args set up
BUCKET_NAME = 'label-tiles'
FOLDER_PATH = 'Sentinel2_tiles_mask/'

sat_name = FOLDER_PATH.split('_')[0]
print('Tiles from satellite:', sat_name)

# GEE Assets args set up
PROJ_NAME = 'nsw-dpe-gee-tst'
ASSETS_SUB_FOLDER = 'OFS' # Can be nested folder, e.g., OFS/sub_folder
IMAGE_COLLECTION_NAME = f'exp_baseOFS_{sat_name}_tiles'

Tiles from satellite: Sentinel2


In [7]:
input_path = os.path.join(os.getcwd(),'drive/MyDrive/JOSS_data/outputs/')

# output_path
output_path = os.path.join(input_path,'json')
if not os.path.exists(output_path):
    os.makedirs(output_path)

# JSON file template
src_file = glob.glob(f'{input_path}Template_{sat_name}_tileset.json')[0]

blobs_lst = list_blobs(BUCKET_NAME, FOLDER_PATH)

upload_lst = [blob for blob in blobs_lst if '.tif' in blob.name]

In [8]:
for blob in upload_lst:
    blob_name = blob.name
    file_name = blob_name.split('/')[1].split('.')[0]
    tile = file_name.split('_')[-1]

    dst_file = f'{output_path}/{sat_name}_EE_upload_{tile}.json'

    shutil.copy(src_file, dst_file)

    assets_name = f'projects/{PROJ_NAME}/assets/{ASSETS_SUB_FOLDER}/{IMAGE_COLLECTION_NAME}/{file_name}'

    source_arg = {
        "uris": [
            f"gs://{BUCKET_NAME}/{blob_name}"
            ]
        }

    tile_arg = {
        "Tile": f"{tile[1:]}" # 'T' needs to be removed
    }

    # Change the base name in the json file
    Write_json(assets_name, dst_file)

    Write_json_Manifest_MultiTiles(source_arg, tile_arg, filename=dst_file)

    # Submit EE Tasks to ingest tiles
    reqID = ee.data.newTaskId()[0]
    with open(f'{dst_file}') as f:
        params = json.load(f)

    ee.data.startIngestion(request_id=reqID, params=params)